In [1]:
import nltk
import random
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import csv
from nltk.corpus import stopwords
import string
import emoji

In [2]:
with open('dataset/apple-sentiment.csv', 'r') as nodecsv: # Open the file                       
    csvreader = csv.reader(nodecsv) # Read the csv  
    # Retrieve the data (using Python list comprhension and list slicing to remove the header row)
    csv = [n for n in csvreader][1:]

In [28]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes 
        \S*
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>')# HTML tags
regex_str.append(r'(?:@[\w_]+)')# @-mentions
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [29]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', '…','•']

In [30]:
for l in stop:
    print(l)

i
me
my
myself
we
our
ours
ourselves
you
you're
you've
you'll
you'd
your
yours
yourself
yourselves
he
him
his
himself
she
she's
her
hers
herself
it
it's
its
itself
they
them
their
theirs
themselves
what
which
who
whom
this
that
that'll
these
those
am
is
are
was
were
be
been
being
have
has
had
having
do
does
did
doing
a
an
the
and
but
if
or
because
as
until
while
of
at
by
for
with
about
against
between
into
through
during
before
after
above
below
to
from
up
down
in
out
on
off
over
under
again
further
then
once
here
there
when
where
why
how
all
any
both
each
few
more
most
other
some
such
no
nor
not
only
own
same
so
than
too
very
s
t
can
will
just
don
don't
should
should've
now
d
ll
m
o
re
ve
y
ain
aren
aren't
couldn
couldn't
didn
didn't
doesn
doesn't
hadn
hadn't
hasn
hasn't
haven
haven't
isn
isn't
ma
mightn
mightn't
mustn
mustn't
needn
needn't
shan
shan't
shouldn
shouldn't
wasn
wasn't
weren
weren't
won
won't
wouldn
wouldn't
!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
:
;
<
=
>
?
@
[
\
]
^
_
`
{
|
}
~


In [31]:
def tokenize(s):
    tokens = tokens_re.findall(s)
    return tokens

In [33]:
def cleanTweet(token,regex):
    terms_all = [emoji.demojize(term) for term in token if term.lower() not in stop and not regex.match(term)]
    return terms_all

In [34]:
tokens = []
for c in csv:
    tokens.append(tokenize(c[0]))

In [35]:
print(tokens[:5])

[['Global', 'warming', 'report', 'urges', 'governments', 'to', 'act', '|', 'BRUSSELS', ',', 'Belgium', '(', 'AP', ')', '-', 'The', 'world', 'faces', 'increased', 'hunger', 'and', '.', '.', '[', 'link', ']'], ['Fighting', 'poverty', 'and', 'global', 'warming', 'in', 'Africa', '[', 'link', ']'], ['Carbon', 'offsets', ':', 'How', 'a', 'Vatican', 'forest', 'failed', 'to', 'reduce', 'global', 'warming', '[', 'link', ']'], ['Carbon', 'offsets', ':', 'How', 'a', 'Vatican', 'forest', 'failed', 'to', 'reduce', 'global', 'warming', '[', 'link', ']'], ['URUGUAY', ':', 'Tools', 'Needed', 'for', 'Those', 'Most', 'Vulnerable', 'to', 'Climate', 'Change', '[', 'link', ']']]


In [44]:
import operator

exclude_str = []
exclude_str.append(emoticons_str)
exclude_str.append(r'<[^>]+>')# HTML tags
exclude_str.append(r'(?:@[\w_]+)')# @-mentions
exclude_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
exclude_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
exclude_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers

exclude_re = re.compile(r'('+'|'.join(exclude_str)+')', re.VERBOSE | re.IGNORECASE)
all_word=[]
from collections import Counter
count_all = Counter()
for token in tokens:
    cleanToken = cleanTweet(token,exclude_re)
    terms_all = [term.lower() for term in cleanToken]
    count_all.update(terms_all)
    for w in terms_all:
        all_word.append(w)
print(count_all.most_common(100))

[('climate', 3422), ('change', 3301), ('global', 3121), ('warming', 3057), ('link', 978), ('new', 334), ('bill', 247), ('snow', 239), ('news', 223), ('science', 197), ('report', 179), ('energy', 179), ('us', 173), ('world', 158), ('à_', 146), ('u', 144), ('dc', 141), ('day', 138), ('could', 136), ('earth', 135), ('obama', 133), ('scientists', 131), ('green', 130), ('al', 120), ('gore', 117), ('conference', 110), ('fight', 109), ('weather', 108), ('un', 108), ('may', 107), ('senate', 106), ('think', 106), ('one', 104), ('legislation', 103), ('says', 101), ('get', 101), ('say', 101), ('time', 101), ('carbon', 96), ('believe', 96), ('com', 95), ('washington', 94), ('help', 94), ('great', 93), ('agency', 91), ('immigration', 90), ('people', 89), ('law', 88), ('winter', 87), ('worse', 86), ('good', 83), ('april', 82), ('action', 81), ('due', 80), ('cold', 80), ('like', 79), ('stop', 78), ('still', 78), ('take', 77), ('study', 76), ('make', 73), ('p', 73), ('makes', 72), ('air', 71), ('debat

In [42]:
text="Photo: Love IOS 8 @apple @iphone #Love #IOS8 #NoCrop #iphone #young #phone #screen #capturescream #boy... http://t.co/fIDj9FxHDV"
token=tokenize(text)
cleanToken=cleanTweet(token,exclude_re)

In [39]:
print(text)

Photo: Love IOS 8 @apple @iphone #Love #IOS8 #NoCrop #iphone #young #phone #screen #capturescream #boy... http://t.co/fIDj9FxHDV


In [40]:
print(token)

['Photo', ':', 'Love', 'IOS', '8', '@apple', '@iphone', '#Love', '#IOS8', '#NoCrop', '#iphone', '#young', '#phone', '#screen', '#capturescream', '#boy', '.', '.', '.', 'http://t.co/fIDj9FxHDV']


In [43]:
print(cleanToken)

['Photo', 'Love', 'IOS', '@apple', '@iphone']


In [45]:
nltk.download('averaged_perceptron_tagger')
pos = nltk.pos_tag(all_word)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
#  j = adjective, r = adverb, n=noun and v = verb
allowed_word_types = ["N","J","V"]
all_adj=[w[0] for w in pos if w[1][0] in allowed_word_types]

In [47]:
allwords = nltk.FreqDist(all_adj)

In [16]:
word_features = list(allwords.keys())[:5000]

In [48]:
print(word_features)

['global', 'warming', 'report', 'urges', 'governments', 'act', 'brussels', 'belgium', 'ap', 'world', 'faces', 'increased', 'hunger', 'link', 'fighting', 'poverty', 'africa', 'carbon', 'offsets', 'vatican', 'forest', 'failed', 'reduce', 'uruguay', 'tools', 'needed', 'vulnerable', 'climate', 'change', 'ocean', 'saltiness', 'shows', 'intensifying', 'water', 'cycle', 'evidence', 'message', 'deniers', 'doubters', 'look', 'migratory', 'birds', 'new', 'strategy', 'stay', 'southern', 'competing', 'limpopo', 'bring', 'higher', 'temperatures', 'southe', 'impact', 'wheat', 'rice', 'production', 'india', 'ludhiana', 'apr', 'scarcity', 'serious', 'solve', 'thing', 'blog', 'preliminary', 'analysis', 'suggests', 'natural', 'gas', 'contribute', 'ecotone', 'perspective', 'blamed', 'coastal', 'whale', 'migration', 'dwindles', 'à_', 'ventura', 'county', 'spring', 'storm', 'season', 'starting', 'little', 'late', 'year', 'government', 'says', 'cause', 'cancer', 'mental', 'illness', 'cnsnews', 'com', 'affec

In [50]:
#document = [(w[0],w[1]) for w in csv]
document=[]
for w in csv:
    categ=""
    if w[1] in ['Yes','Y']:
        categ="pos"
        document.append((w[0],categ))
    if w[1] in ['No','N']:
        categ="neg"
        document.append((w[0],categ))

In [51]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [52]:
featuresets = [(find_features(rev), category) for (rev, category) in document]

In [53]:
print(featuresets[:1])

[({'global': False, 'warming': True, 'report': True, 'urges': True, 'governments': True, 'act': False, 'brussels': False, 'belgium': False, 'ap': False, 'world': True, 'faces': True, 'increased': True, 'hunger': True, 'link': True, 'fighting': False, 'poverty': False, 'africa': False, 'carbon': False, 'offsets': False, 'vatican': False, 'forest': False, 'failed': False, 'reduce': False, 'uruguay': False, 'tools': False, 'needed': False, 'vulnerable': False, 'climate': False, 'change': False, 'ocean': False, 'saltiness': False, 'shows': False, 'intensifying': False, 'water': False, 'cycle': False, 'evidence': False, 'message': False, 'deniers': False, 'doubters': False, 'look': False, 'migratory': False, 'birds': False, 'new': False, 'strategy': False, 'stay': False, 'southern': False, 'competing': False, 'limpopo': False, 'bring': False, 'higher': False, 'temperatures': False, 'southe': False, 'impact': False, 'wheat': False, 'rice': False, 'production': False, 'india': False, 'ludhian

In [54]:
# menyusun data training dan testing 
trainsize = round(len(featuresets)*0.7)

random.shuffle(featuresets)
training_set = featuresets[:trainsize]
testing_set = featuresets[trainsize:]

In [58]:
sum(p[1]=='neg' for p in training_set)

769

In [59]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(25)

Classifier accuracy percent: 76.87450670876085
Most Informative Features
                 hearing = True              neg : pos    =     16.1 : 1.0
                 control = True              neg : pos    =     16.1 : 1.0
                imminent = True              neg : pos    =     14.2 : 1.0
                   snows = True              neg : pos    =     14.2 : 1.0
                disaster = True              neg : pos    =     12.3 : 1.0
                  shovel = True              neg : pos    =     12.3 : 1.0
                happened = True              neg : pos    =     12.3 : 1.0
               alarmists = True              neg : pos    =     12.3 : 1.0
                      al = True              neg : pos    =     11.9 : 1.0
                    hoax = True              neg : pos    =     11.8 : 1.0
                    gore = True              neg : pos    =     10.8 : 1.0
                   stuff = True              neg : pos    =     10.4 : 1.0
               shoveling = 

In [24]:
from nltk.metrics.scores import (precision, recall)
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print('Recall:', recall(refsets['pos'], testsets['pos']))
print('Precision:', precision(refsets['pos'], testsets['pos']))

Recall: 0.8728260869565218
Precision: 0.8373305526590198


In [41]:
classifier2 = nltk.classify.SklearnClassifier(LinearSVC())
classifier2.train(training_set)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [42]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier2, testing_set))*100)

Classifier accuracy percent: 81.29439621152328


In [43]:
classifier3 = SklearnClassifier(LogisticRegression(solver='lbfgs')).train(training_set)

In [44]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier3, testing_set))*100)

Classifier accuracy percent: 81.05761641673243


In [45]:
for (rev, category) in document[trainsize:]:
    result=classifier2.classify(find_features(rev))
    print(rev,"-",category,"-",result)

US Senate CAP & TAX *** Bill to be unveiled April 26 *** Global Warming Swindle (Video's too)|Our work is cut out .. http://oohja.com/xb3rj - neg - neg
Algae for controlling global warming - Current News & Events ... http://bit.ly/ax6lEp - pos - pos
Addressing Global Warming: Retrain Coal Workers for Green Jobs ... http://bit.ly/cW1KTV - pos - pos
Forests Are Growing Faster,Climate Change Appears to Be Driving Accelerated Growth (Science Daily) http://bit.ly/d1EqlN @InvasiveNotes - pos - pos
Obama's base cares about their religion of global warming and Marxism. The myth of healthcare. The myth of peace without having to be strong - neg - neg
Global warming may become global cooling this century http://bit.ly/cU8D4F - pos - pos
Global Warming: Ozone Hole Healing http://bit.ly/aXpdcl - pos - pos
My daughter says Global Warming Is NOT Cool!  http://twitgoo.com/ok5ya - pos - pos
Earth's polar ice sheets vulnerable to even moderate global warming; New Orleans, much of southern F... http://c

Media cools on global warming: By News on the Net Saturday, February 6, 2010 LAST weekend looks likely to have bee... http://bit.ly/9d2s2H - neg - neg
Claiming Climate change is man made or it will effect our lives if we dont change our ways IS COMPLETE BULLSHIT! TREE HUGGERS HARM US MORE! - neg - neg
http://bit.ly/bgaWIH An Honest IPCC Scientist Warns His Colleagues: Dont Dismiss ClimateGate ways to stop global warming :P - pos - pos
RT @BarackBorat: Can someone call the Al Gore and ask him for make turn the Global Warming back on? - neg - neg
New scientific consensus: Hot air from global warming alarmists is melting Himalayan glaciers! Journalists are free to cite this tweet.#hhrs - pos - pos
Yeah...global warming my ass... - neg - neg
@LaurenConrad Oh.  Well, there goes the global warming theory LOL - neg - neg
@edenjames sign of the apocolypse: winter. Evidence against global warming: winter. Brr boo brrr - neg - neg
ENVIRONMENTAL REPUBLICAN: Global Warming Belief Melting Away: The

Global warming research grant in danger of fund freeze: By Stephen Dinan A top member of Congress wants President ... http://bit.ly/aAY4xX - pos - pos
RT @ThePhoenixSun RT @ejgertz: Florida Gov Agencies on Global Warming: Red Alert! http://bit.ly/chyz25 #climate #FL #eco - pos - neg
Check this out - the cost of climate change: http://www.nrdcactionfund.org/thisisourmoment/?p=1281&c=349598&s=762984&l=81599 - pos - pos
RT @thedailybeast: GOP pollster Frank Luntz who counseled Bush not to buy into the science on global warming is changing his view. - pos - pos
Water Vapor Slowed Global Warming Over Past Decade: Over the last 10 years, global temperatures rose about 25 pe... http://bit.ly/b5dfKU - pos - pos
@AFPHQ Americans For Prosperity Praises Key Democrats for Joining Effort to stop EPA Global Warming Power Grab  http://twurl.nl/dbtyr4 - pos - pos
"The global warming movement as we have known it is dead,"writes Walter Russell Mead of the Council on Foreign Relations in The American Int

Global Warming & Our Winter Storms http://www.paltalknewsnetwork.com/node/2210 #globalwarming #weather - pos - pos
I lovce hearing "weather isnt climate" from the same idiots that blamed Katrina on "climate change". - neg - neg
RT @change The Geological Society of America rejects global warming theory & member speaks out http://bit.ly/cGFw4o (via @rebeccawalker) - neg - pos
European Forests and Carbon Sequestration Services: An Economic Assessment of Climate Change Impacts - http://bit.ly/b8ifs9 - pos - pos
Im up. Still snowing out there. Dear Al Gore, bite me. Oh wait, your claimign this is now caused by "climate change" right. Ass. - neg - neg
Global Warming lol! Al Gore you need 2 step up your theory brother. This season produced the most snow I seen in life. Evrywk it is snowin - neg - neg
D.C. Snowstorm: How Global Warming Makes Blizzards Worse: There's scarcely any powder in Vancouver, while Baltimor... http://bit.ly/aLa47B - pos - pos
@mimidancer just because it also gets colder

Maybe the second DC snowstorm in a week is the universe sending climate change naysayrers in the gov't a yes vote? - neg - pos
Story of the vaporization of the great gobal warming scandal #global warming http://is.gd/7TByc - neg - neg
RT @wsmco: if you're one of the 10s of thousands without power from the blizzard and you voted for this global warming dipshit, i'm glad you're f-d. - neg - neg
Climate-change legislation buried under record snowfall|Record snowfall has buried Washington ? and along with .. http://oohja.com/x6IMs - neg - neg
Liked "New Climategate Shocker: Even the CRU Thought the Climate Change Hockey Stick Was Flawed: The term "hockey..." http://ff.im/-fGwfE - neg - neg
From DailyKos: Climate Change Skeptics Discredited - New information suggests that a report criticizing data for cl... http://ow.ly/16xz87 - neg - neg
ha! RT @CrabbyCon: Global Warming mockery video that includes recent photos of the Great Blizzard of 2010 http://bit.ly/942I4T #tcot - neg - neg
green and

...

Our climate is changing just as we are changing, do you believe in climate change? - pos - pos
RT @AmazonWatch: People's Conference on Climate Change & Rights of Mother Earth opening now' Watch LIVE: http://bit.ly/dsYZUV - pos - pos
Climate Scientists Cleared, But Not U.N.: Scientists in climate scandal made no attempt to change global warming r... http://bit.ly/9Tml0g - neg - neg
Check out Climate Desk, an interesting new collaboration dedicated to reporting on climate change:  http://bit.ly/dAsx5m - pos - pos
RT @GreenHour: BudBurst volunteers needed: to look for "ten most wanted" plants to assess clues to climate change: http://bit.ly/bdlpVW - pos - pos
RT @VRWCTexan: STATE DEPT DECLARES: Global warming unequivocal & primarily human-induced' Vs. "Poll:4 of 5 Americans don't trust Washington" - pos - neg
RT @Uncucumbered: U know wht wld B fun? If the Sierra Club announced guns caused global warming. Talk abt yr ensuing Govertainment. - neg - neg
Does this mean they're sure? . .STATE 

Watched British debates on CSPAN. Frightening. Every candidate is a leftist, including those on the "right." I.e., global warming is TRUTH. - pos - pos
Geologists Study Historic Patterns of Climate Change: University of Cincinnati geologist Tom Lowell is part of a t... http://bit.ly/cqO8E5 - pos - pos
Is global warming a real problem or a hoax?: by Mark Hopkins Global warming is a real problem but so is proving be... http://bit.ly/9Hy51b - pos - neg
Giles Slade: McCarthyism and Climate Change: Is it strange that Sarah Palin, who once thought Africa was a countr... http://huff.to/di5DUA - pos - pos
Global warming: a market proposal even libertarians would love: Christian Science Monitor Libertarians may debate ... http://bit.ly/brZ7Zz - neg - neg
How does co2 cause global warming? | Global Warming Cause: How does co2 cause global warming? why specifically is ... http://bit.ly/9L5JlS - pos - pos
US Government's New Climate Change Claim: Global Warming Causes ...: Before I begin, I promis

RT @GOOD: The good news about the very bad news (about climate change) http://su.pr/1fG7eV - pos - pos
The good news about the very bad news (about climate change) http://su.pr/1fG7eV - pos - pos
RT @latimes: Ice-covered volcanoes may answer climate change questions http://lat.ms/9uVJRK - pos - pos
Ice-covered volcanoes may answer climate change questions (I thought it was all about SUVs) http://ow.ly/1CzFa - neg - pos
RT @G_Lauricella: Is Climate Change Genocide? http://bit.ly/asmJGu - pos - pos
The Good News About the Very Bad News (About Climate Change) http://ow.ly/17a4YA - pos - pos
Global Warming Causes Earthquakes.' Suuuuure it does.: As [the glaciers] flow off the land, we are warned, seas r... http://bit.ly/c7OhVu - neg - neg
via @huffingtonpost: News Corp Aims To Go Carbon Neutral While Fox News Promotes Climate Change As... http://huff.to/dvvVs8 - pos - pos
Caf•__ Scientifique presents global warming talk, film: The April Caf•__ Scientifique presentation will be on the topi.

@Qorianka Many thanks to you Sister,for your partcipation in the World Peoples Summit for Climate Change. #Indigenous #EarthRights #Global - pos - pos
Climate change: http://tinyurl.com/rockclimbpolarbear -- he's a cutie; evolution is already starting..slowly. #polarbears - pos - pos
New Global Warming Alarmism from LA Times: 'The World is Running Short on Air ...: We have now reached the apex of... http://bit.ly/chaIBp - pos - pos
One thing is climate change and other different is global warming caused by CO2 increase in the atmosphere. http://bit.ly/aXgVNi - pos - pos
Big Ups to the ppls at the World People's Conference On Climate Change in Bolivia  #EarthWeek #EarthDay - pos - pos
I liked a YouTube video -- Feel-Good Fantasies of Fighting Global Warming http://youtu.be/l4f57whozrY?a - neg - neg
Skeptic's Corner: Voters Take Global Warming A Bit Less Seriously: FROM-Rasmussen Report Monday, April 19, 2010. V... http://bit.ly/bqm77F - neg - pos
BRKG NEWS: UN Climate Change panel gets 

RT @earthvitalsigns: Ice ice baby! Get the lowdown on ice & climate change with the next in our series of Just 5 Questions... http://bit.ly/c6YR9V - pos - pos
Ice ice baby! Get the lowdown on ice & climate change with the next in our series of Just 5 Questions... http://bit.ly/c6YR9V - pos - pos
Natural Resource Econ: Krugman on Climate Change http://bit.ly/b4IyRj - pos - pos
Natural Resource Econ: Krugman on Climate Change http://bit.ly/b4IyRj - pos - pos
OXFAM GB: Senior Press Officer - Climate Change: •_£33,700 - •_£41,710 pa: OXFAM GB For more Charities jobs  and jobs... http://bit.ly/bx6C5j - pos - pos
THE best climate change economics primer. - jasencomstock: I love Paul Krugman, and I have to say he makes an... http://tumblr.com/xkw8jdz7y - pos - pos
RT @JerryMander: Daily Kos: ecojustice: systems change, not climate change & NOT violence! http://bit.ly/bBxEKB | "Right now, our elected - pos - pos
Bonn to Cancun'Negotiators Agree to Continue Efforts on International Global ...: 